In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import walk
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D
from keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

Using TensorFlow backend.


In [2]:
cat = ['bedroom', 'CALsuburb', 'coast', 'forest', 'highway', 'industrial', 'insidecity', 'kitchen', 'livingroom', 'mountain', 'opencountry', 'PARoffice', 'store', 'street', 'tallbuilding']  

In [3]:
num_classes = len(cat)

In [4]:
trainpath="/data/examples/may_the_4_be_with_u/where_am_i/train"
testpath="/data/examples/may_the_4_be_with_u/where_am_i"

In [5]:
img_total = []
label = []
for i in range(len(cat)):
    f = []
    for (dirpath, dirnames, filenames) in walk(trainpath +'/{}'.format(cat[i])):
        f.extend(filenames) 
    print(len(f))
    for j in range(len(f)):
        imgp = trainpath +'/{}/{}'.format(cat[i], f[j])
        img = image.load_img(imgp, target_size=(224, 224), grayscale = True)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img/255
        img_total.append(img)
        label.append(cat[i])

116
141
260
228
160
211
208
110
189
274
310
115
215
192
256


In [6]:
img_train = np.array(img_total)
img_train = img_train.reshape(2985, 224, 224, 1)

In [7]:
x_train = img_train

In [8]:
label = np.array(label)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(label)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_TrainOneHot = onehot_encoder.fit_transform(integer_encoded)


In [9]:
model = Sequential()
model.add(Conv2D(64, (5, 5), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(128, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (5, 5), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [10]:
opt = keras.optimizers.Adam(lr=0.0001)
#opt = keras.optimizers.Adam()
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True, write_images=True)

In [12]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(img_train)

In [13]:
model.fit_generator(datagen.flow(x_train, y_TrainOneHot,batch_size=32),
                    steps_per_epoch=x_train.shape[0] // 32,
                    epochs=130,
                    validation_data=None, 
                    callbacks=[tbCallBack])

Epoch 1/120
93/93 [==============================] - 32s 347ms/step - loss: 2.6535 - acc: 0.1077
Epoch 2/120
93/93 [==============================] - 28s 296ms/step - loss: 2.4096 - acc: 0.1981
Epoch 3/120
93/93 [==============================] - 27s 296ms/step - loss: 2.2749 - acc: 0.2610
Epoch 4/120
93/93 [==============================] - 28s 298ms/step - loss: 2.1631 - acc: 0.2849
Epoch 5/120
93/93 [==============================] - 27s 295ms/step - loss: 1.9958 - acc: 0.3528
Epoch 6/120
93/93 [==============================] - 28s 298ms/step - loss: 1.8278 - acc: 0.4048
Epoch 7/120
93/93 [==============================] - 28s 298ms/step - loss: 1.7066 - acc: 0.4271
Epoch 8/120
93/93 [==============================] - 27s 295ms/step - loss: 1.6406 - acc: 0.4502
Epoch 9/120
93/93 [==============================] - 27s 294ms/step - loss: 1.5704 - acc: 0.4803
Epoch 10/120
93/93 [==============================] - 28s 296ms/step - loss: 1.5020 - acc: 0.4947
Epoch 11/120
93/93 [=========

In [14]:
img_test = []
f = []
pred = []
for (dirpath, dirnames, filenames) in walk(testpath +'/testset'):
    f.extend(filenames) 
for j in range(len(f)):
    imgp = testpath + '/testset/{}'.format(f[j])
    img = image.load_img(imgp, target_size=(224, 224), grayscale = True)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img/255
    p = model.predict(img)
    pred.append(p)

In [15]:
k = [np.argmax(r) for r in pred]

In [16]:
dic = {0:9,
      1:7,
      2:12,
      3:10,
      4:4,
      5:14,
      6:2,
      7:3,
      8:0,
      9:5,
      10:8,
      11:6,
      12:11,
      13:1,
      14:13}

In [17]:
cla = []
for i in range(len(k)):
    cla.append(dic[k[i]])

In [18]:
a = []
for i in range(len(f)):
    a.append(f[i][:-4])

In [19]:
sub = pd.DataFrame(a)

In [20]:
sub['class'] = cla

In [21]:
sub.columns = ['id','class']

In [22]:
sub.head()

,id,class
0,7dc74b51e229d841272c0795cffed857d0e6038a4be0c9...,7
1,1b1940d44fe4f5b76e89f876de3d2514a51b50057cae3e...,3
2,7cb3479dac3e7dcc69241f4cd957380a48399b1aa0480d...,3
3,dbbcb75061f5a6ac493af0fa87ce806977a0a4cebce9b7...,3
4,d3238e3700b076099b4fad94dc1714d2bf14c7aedb2e24...,13


In [23]:
sub.to_csv('submit_16.csv', index=False)

In [95]:
model.save('s11.h5')